In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
import tensorflow_hub as hub
import tensorflow as tf

class SmallDataset(Dataset):
    """A simple dataset for demonstration purposes."""
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label

def load_model_from_hub(url, save_path):
    """
    Load a model from TensorFlow Hub, save it locally, and return a PyTorch model.
    """
    # Load the TensorFlow Hub model
    model = hub.load(url)
    
    # Save the model locally
    tf.saved_model.save(model, save_path)
    print(f"Model saved to {save_path}")

    return model

def inspect_model(model):
    """
    Inspect the model's architecture and parameters.
    """
    model.summary()  # This works for TensorFlow models
    for layer in model.layers:
        print(f"Layer: {layer.name}, Output shape: {layer.output_shape}, Trainable: {layer.trainable}")

def convert_to_pytorch(model):
    """
    Extract the core model architecture from TensorFlow Hub and use it in PyTorch.
    """
    # Extract the feature layers from the TF model (remove final classification layer)
    feature_extractor = model.signatures['serving_default']
    
    # Assuming we get embeddings as output, this would be the feature extractor
    return feature_extractor

class TransferLearningModel(nn.Module):
    """
    Custom PyTorch model for transfer learning.
    """
    def __init__(self, feature_extractor, num_classes=13):
        super(TransferLearningModel, self).__init__()
        self.feature_extractor = feature_extractor  # Pre-trained feature extractor
        self.fc = nn.Linear(1280, num_classes)  # Adjust this based on the output size of your extractor

    def forward(self, x):
        with torch.no_grad():
            # Extract features from the pre-trained model
            features = self.feature_extractor(x)
        # Pass through the new classification layer
        x = self.fc(features)
        return x



In [4]:
def freeze_parameters(model):
    """
    Freeze all parameters in the feature extractor and only keep the final layer trainable.
    """
    # Freeze all layers except the final fully connected layer
    for param in model.feature_extractor.parameters():
        param.requires_grad = False
    print("All parameters frozen except the final layer.")

def train_model(model, dataloader, num_epochs=5, learning_rate=0.001):
    """
    Train the modified model on a very small dataset.
    """
    # Use cross-entropy loss for classification
    criterion = nn.CrossEntropyLoss()
    
    # Only optimize the final layer
    optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
    
    # Training loop
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            # Move inputs and labels to GPU if available
            inputs, labels = inputs.cuda(), labels.cuda()
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}")
    
    print("Training complete.")

def prepare_dataloader(data, labels, batch_size=4):
    """
    Prepare a DataLoader for the small dataset.
    """
    transform = transforms.Compose([transforms.ToTensor()])
    dataset = SmallDataset(data, labels, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import os
import numpy as np


In [9]:
# Define model input
input_waveform = tf.keras.Input(shape=(16000,), dtype=tf.float32, name='waveform')
print(input_waveform.shape)

save_path = r'C:\Users\rapha\repositories\guitar_hero\vggish_model\saved_model.pb'

(None, 16000)


In [11]:
vggish_layer = hub.KerasLayer('https://tfhub.dev/google/vggish/1', trainable=False)
print("ok")
input_waveform = tf.keras.Input(shape=(16000,), dtype=tf.float32, name='waveform')
print("ok2")
vggish_outputs = vggish_layer(input_features)
print("ok3")
model = tf.keras.Model(inputs=input_features, outputs=vggish_outputs)

ok
ok2


TypeError: Exception encountered when calling layer 'keras_layer_1' (type KerasLayer).

Binding inputs to tf.function failed due to `A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
`. Received args: (<KerasTensor shape=(None, 96, 64), dtype=float32, sparse=False, name=input_features>,) and kwargs: {} for signature: (waveform: TensorSpec(shape=(None,), dtype=tf.float32, name=None)).

Call arguments received by layer 'keras_layer_1' (type KerasLayer):
  • inputs=<KerasTensor shape=(Non

In [8]:
loaded_model = tf.keras.models.load_model(save_path, custom_objects={'KerasLayer': hub.KerasLayer})
print(f"Model successfully loaded from {save_path}")
loaded_model.summary()
return loaded_model

ValueError: File format not supported: filepath=C:\Users\rapha\repositories\guitar_hero\vggish_model\saved_model.pb. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(C:\Users\rapha\repositories\guitar_hero\vggish_model\saved_model.pb, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:

# Load the pre-trained TensorFlow model from hub
#tf_model = load_model_from_hub('https://tfhub.dev/google/vggish/1', save_path='./vggish_model')

# Load the pre-trained TensorFlow model from disk

# Convert to PyTorch compatible model
pytorch_model = convert_to_pytorch(tf_model)

# Create a PyTorch transfer learning model
model = TransferLearningModel(pytorch_model, num_classes=13)

# Freeze the feature extractor layers
freeze_parameters(model)

# Prepare a small dataset for training
data = [...]  # Your data here (could be images, spectrograms, etc.)
labels = [...]  # Corresponding labels
dataloader = prepare_dataloader(data, labels, batch_size=4)

# Train the model
train_model(model, dataloader, num_epochs=5)